In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello World : Votre premier circuit quantique

Construisez un [état de Bell](https://en.wikipedia.org/wiki/Bell_state) (deux qubits intriqués) et exécutez-le de trois manières :

1. **Simulation idéale** — résultats parfaits, aucun compte requis
2. **Simulation bruitée** — simule un véritable appareil, aucun compte requis
3. **Matériel quantique réel** — nécessite un [compte IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## Construire le circuit

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## Option 1 : Simulation idéale (aucun compte requis)
Utilise `StatevectorSampler` — un simulateur local avec des résultats parfaits, sans bruit.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## Option 2 : Simulation bruitée (aucun compte requis)
Utilise `FakeManilaV2` — un simulateur local qui reproduit le comportement d'un véritable appareil quantique IBM, y compris ses caractéristiques de bruit. Le circuit doit d'abord être transpilé (adapté) au jeu de portes et à la connectivité des qubits de l'appareil.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Option 3 : Matériel quantique réel
Nécessite un compte IBM Quantum. Consultez [Configurer votre compte IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) pour plus de détails.

Si vous n'avez pas encore enregistré vos identifiants dans cette session Binder, exécutez d'abord ceci :

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**Remarque :** Les tâches sur du matériel réel peuvent prendre un certain temps en fonction de la file d'attente. Si la cellule est toujours en cours d'exécution, vous pouvez vérifier l'état de votre tâche et consulter les résultats sur [quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me).

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Et ensuite ?
- **[Tutoriels](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — guides pas à pas sur les algorithmes, l'atténuation des erreurs, la transpilation et bien plus encore
- **[Cours](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — parcours d'apprentissage structurés, des bases du quantique au calcul à grande échelle
- **[Mode de test local](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — exécutez la plupart des notebooks sans compte IBM Quantum